# Sequence Data
## Time Series
The first thing that come to people minds when asked to think of a sequence is time series, which represents any continuous-valued measurement taken periodically. ex: a company's stock price, weather forcasting
### Example Airline Passangers
If you can forecast the number of passengers you will have on your airline it will help you:
- Make sure the airport has enough workers to handle the passengers
- Modify prices preemptivly to reflect demand (e.g. Uber)
- Decide where to spend resources and marketing

## Text
Another type of sequential data is text, the intresting thing that you don't have to treat text as sequential data, in the olden days of ML dealing with sequncies was a bit hard so experts tried to use more naieve approach called `bag-of-words` feature vector.
### Bag of words
Suppose you are doing document classification (email -> spam vs. not spam), you would have to create a big long feature vector (one entry for each word in the English language), inside the vector you insert the count of how many times each word appeared in the email.
- Suppose there are only 5 words in the English Language: (insurence, loan, pickles, backpack, football)
- The email contians `insurance` 3 times, `loan` 1 time, and the other words 0 time
- The `bag-of-words` vector will be (3,1,0,0,0)
- The lables will be (0 or 1) represent the status of the email
#### Problems with Bag of words
When you only count each word, you will lose information about the order (`Dog toy` vs `Toy dog`) not only the order is different but the meaning is also different

## Shape of a sequence
When you think about it you cannot represent the length of a sequence niether by `N = #samples` nore `D = #features`, so the need for a new letter has arraised `T = #time steps in the sequence` which is intuitive because the word `time` start with it. Our data is represented as a 3-D array of size `NxTxD`.
### Example 1
Suppose we want to model the path employees take to get to work, so you record GPS data from their cars, can you tell what is N? D? T? in this case.
- N: One sample would be one person's single trip to work<br />
  * The dataset may contain multiple trips from the same person - those would be separate samples
- D: The GPS will record (latitude, longitude) pairs
- T: The number of `(lat, lng)` measurements taken from start to finish of a single trip
  * E.g. trip is 30 min, coordinates are measured every second `(T = 30 * 60 = 1800)`
Here a problem will arrise each person take a diiferent  amount of time to get to work so the T value will vary and this is against Tanserflow/Keras neature `equal-length sequences`
### Example 2
A stock price is just a single value so `D=1`, as a simple example suppose we use a window of size `T=10` to predict the next value then `N = the number of windows in the TS` so the mathimatics will be:

If you have a sequence of 100 stock prices, how many windows of length 10 are there? `(100 - 10 + 1 = 91),(L - N + 1 )`
If we measure the stock price every day for 500 different stocks then `D=500`, then a single sample will be of size `TxD = 10x500`

## Variable length sequences
It is a pretty common scenario - e.g. sentences `not all sentences have the same number of words`, in the past experts used variable length sequences which is very complicated to work with `(Inefficient data structures)`.

In PyTorch, it is easier to start with constant length sequences by padding each sequence shorter than the longest sequence with zeros, it will lead to consume unneccery memory as zeros will consume it, but in the other hand it does make writing code easier.
PyTorch dose padding better by implementing the largest sequence size based on the largest sequence in the currnet `patch` not in the whole list.

# Forecasting
Many researchs do this in a way that looks nice, but results don't make sense, in our context forecast means to predict the next `values` of a `TS`.
We don't usually want to predict just one step ahead `(problematic)`, the Number of future steps we want to predict is `horizon`, E.g.1 Predict demand for 3-5 days for products manufactured in factory, E.g.2 Weather - hourly weather for 7 days `(7 X 24 = 168)`.

# Recurrent Neural
Previously, we’ve seen that simple data that can be expressed as a vector can be easily provided to the input layers of a feedforward neural network. More complex data like images can either be `transformed and flattened` to be sent as input in vector form or `can be used to learn filters` in a convolutional neural network (CNN).
CNNs helped capture essential patterns in the data that occur due to certain values present in the proximity of a pixel. However, there’s another pattern that usually occurs in data formats like text, speech, etc., as shown below:

![rnn-text-seq](images/rnn-text-seq.png)

In sequential data formats like these, it can be observed that in terms of steps of time (say, based on time slice in audio, or character by character in text), the patterns between the data at the current step and the previous step(s) lead to a different meaning if the data in one of the steps changes.

Recurrent neural networks (RNNs) are a type of neural networks that are specialized for processing a temporal sequence of data. In this section, we will learn about the structure of the basic unit of the RNNs and how they fit in a larger neural network. We’ll also study specialized RNN units like LSTM that have broken records for machine translation, text-to-speech generation, speech recognition, and other tasks.

## Recurrent Unit
Recurrent neural networks accept the provided chunk of input data and use an internal state to produce the output. The internal state is calculated from the old internal state and the input – thus, we can say that indirectly, an output of the recurrent cell in the neural network in a previous step determines the output in the current step. In a way, if we understand it in this way, recurrent neural networks get part of their output as input for the next time step.

Assume that the data obtained at a particular timestamp is present in the form of D-dimensional vectors. At time step index 1, x(1) is the input vector of size `D`, followed by x(2) in time step index 2, and so on, up to x(T) at time step `T`. This represents one sample of the dataset in the form of a `TxD` shape matrix as shown in below figure. For `N` samples like these, you can assume the data in the form of an `NxTxD` shape matrix as shown in below figure also.

![rnn-text-as-v-across-time](images/rnn-text-as-v-across-time.png)

At the simplest level, input at each unit is applied to a weighted function as you sawin the previous chapters. These weights are shared through time and applied to each input, along with the output of the previous step. This operation can be expressed as shown in below.

![unfolding-rnn-cell](images/unfolding-rnn-cell.png)

The portion on the left shows that the input x passed to the recurrent unit cell applied with the weights W leads to the output o. When this recursion is unfolded over time as shown in the portion on the right, you can see that there are three time steps, each with inputs `Xt-1`, `Xt`, and `Xt+1`, respectively. At step `t`, the operation that occurs in the hidden layer can be expressed as:

Func

The expansion, or unfolding, thus done over time leads to a network that now looks similar to a feedforward network with multiple hidden layers, each representing the operations conducted at one time step. This expansion depends on sequence length and how many such combinations, given by batch size. You can think of unfolded structure as a feedforward network. During the training phase, our aim is to learn the weights by computing the loss based on the prediction with respect to the actual output label and accordingly compute the gradients for backpropagation, which will adjust the weights to produce a better prediction in the next iteration.

If we unroll all input time steps, we extend the network to a structure with inputs at each time step leading to the output via hidden layers with common weights. Each time step in this technique can be seen as an extension of a hidden layer, with the previous time step’s internal state serving as an input. Each time step has one input time step, copy of the network, and one output. Errors are then calculated and accumulated for each time step. The network is rolled back up, and the weights are updated. This is called backpropagation through time (BPTT).

We’ll observe that due to the chain rule, we will often see similar multiplication operations on very small values happening over and over again, which may cause the result to approach 0 or infinity. We will discuss this issue and a common potential solution later.

## RNN in Python
PyTorch provides an RNN implementation that requires the specifications of your recurrent layer and can be added as part of a larger network. It applies the weights and bias to the input data and combines that with the weighted hidden statuses and finally applies a nonlinear function (ReLU or tanH) to that. This provides the hidden state for the next input.

Let’s begin with a simple example that takes a sequence of numbers. Let’s begin with importing the required libraries. RNN is defined under `torch.nn`.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

Let’s create a simple sequence as a tensor.

In [ ]:
data = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
print("Data: ", data.shape, "\n\n", data)

To define a simple RNN, we need to define sequence length, batch size, and input size. In this example, we will predict the next two numbers based on five contiguous numbers of the sequence.

We have simple data with only one column, and the problem we’re trying to solve says that we have to observe a sequence of length of five inputs to predict the next two outputs.

In [ ]:
INPUT_SIZE = 1
SEQ_LENGTH = 5
HIDDEN_SIZE = 2
NUM_LAYERS = 1
BATCH_SIZE = 4

In most cases, one or more layers of RNN will be part of a larger network that ends with an output layer. Based on the hyperparameters that specify how the layer in the network should look like, we can define the RNN layer using:

In [ ]:
rnn = nn.RNN(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers=1, batch_first=True)

We can use the data as input and obtain the output and the hidden state using the following. The size of the input depends on batch size, sequence length, and input size – the parameters we defined previously. nn.RNN() can also accept pre-set values for hidden layer, which default to all zeros if not specified.

In [ ]:
inputs = data.view(BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)
out, h_n = rnn(inputs)

The output we thus receive is the tensor containing output of the RNN from all time steps from the last RNN layer. It’s size is (sequence length, batch, num_directions * hidden size), where num_directions is 2 for bidirectional RNNs, otherwise 1.